Piiranha =
├── Base model (language understanding)
│   └── produces vectors for each token
├── Token-classification head
│   └── converts vectors → label scores
├── Trained weights
│   └── learned during PII training
└── Config
    └── defines labels (PERSON, EMAIL, etc.)

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")
# We import torch because the model runs on PyTorch, and PyTorch is the engine that actually does the math.
import torch

# transformers is a Python library made by Hugging Face.It contains Pretrained language models (BERT, RoBERTa, etc.), Tokenizers and utilities to load model from the Hugging Face Hub.
# A tokenizer that automatically knows how to tokenize text for a given model. you do not need to say it which model you are using, just give it the model name and it will do the rest looking up the model config from the Hub.
# AutoModelForTokenClassification Loads a pretrained model for token-level tasks (like PII or NER) with the token-classification head already attached. It reads the model’s config.json to know the task, labels, and output shape, and loads all the trained weights (both the base model and the head) from the model folder, so it’s ready to predict labels for each token without any extra setup.
from transformers import AutoTokenizer, AutoModelForTokenClassification

MODEL_ID = "iiiorg/piiranha-v1-detect-personal-information"
LOCAL_MODEL_DIR = "models_store/piiranha"

if os.path.isdir(LOCAL_MODEL_DIR) and os.listdir(LOCAL_MODEL_DIR):
    print(f"Loading model from local directory: {LOCAL_MODEL_DIR}")
    tokenizer = AutoTokenizer.from_pretrained(LOCAL_MODEL_DIR)
    model = AutoModelForTokenClassification.from_pretrained(LOCAL_MODEL_DIR)

else:
    print(f"Downloading model from Hugging Face Hub: {MODEL_ID}")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
    model = AutoModelForTokenClassification.from_pretrained(MODEL_ID, token=HF_TOKEN)

    tokenizer.save_pretrained(LOCAL_MODEL_DIR)
    model.save_pretrained(LOCAL_MODEL_DIR)

text = "Can you confirm my phone number 9876543210?"

# Tokenize text
# return_tensors="pt". This model is a PyTorch model, so we want the tokenizer to return PyTorch tensors, not plain python lists.
# truncation=True. This ensures that if the input text is longer than the model's maximum input length, it will be truncated to fit.
# The context length is 256 Deberta tokens(sub words), so approximately 160 - 200 words.
inputs = tokenizer(text, return_tensors="pt", truncation=True, is_split_into_words=False)
print("Tokenized input IDs:", inputs["input_ids"])
print("Tokenized tokens:", tokenizer.convert_ids_to_tokens(inputs["input_ids"][0]))

# Forward pass (get logits)
# Normally, PyTorch keeps track of all calculations so it can compute gradients for training. Since we are only doing inference here, we disable gradient tracking with torch.no_grad() to save memory and computation.
# inputs is the tokenized sentence, which includes things like input IDs and attention masks.
# Attention mask tells the model which tokens are real text (1) and which are padding (0).
# If a sequence is shorter than max_length, the tokenizer pads it with zeros and marks those
# padded positions with 0 in the attention mask so the model ignores them during attention.
# The model processes the input and returns outputs, which include logits (raw vectors before applying argmax).
with torch.no_grad():
    outputs = model(**inputs)

#logits are raw predictions from the model, before converting them to readable labels.
#logits is a matrix of numbers, one row per token, one column per label.
logits = outputs.logits
print("Model outputs:", outputs)
print("Logits:", logits)
print("Logits shape:", logits.shape)  # (batch_size, num_tokens, num_labels)

# Convert logits to predicted label IDs
#argmax finds the index of the largest number along a specified dimension.
#dim=-1 → look across the last dimension, which is labels for each token.
#Tensor shape: (batch_size, num_tokens, num_labels)
pred_ids = torch.argmax(logits, dim=-1)
print("Predicted label IDs:", pred_ids)

# Map label IDs to actual label names
id2label = model.config.id2label
pred_labels = [id2label[i.item()] for i in pred_ids[0]]
print("Predicted labels for each token:", pred_labels)

# Map tokens to words (manual aggregation)
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
for token, label in zip(tokens, pred_labels):
    print(f"{token:15} -> {label}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Tokenized input IDs: tensor([[     1,   4017,    522,  17687,   1038,  11348,   4404,    260, 232825,
         231638,    661,    292,      2]])
Tokenized tokens: ['[CLS]', '▁Can', '▁you', '▁confirm', '▁my', '▁phone', '▁number', '▁', '9876', '5432', '10', '?', '[SEP]']
Model outputs: TokenClassifierOutput(loss=None, logits=tensor([[[ 7.1680e-01, -9.5567e-01, -2.3313e+00, -2.5904e-01, -6.7706e-01,
          -8.8575e-01, -1.0911e+00,  5.2335e-01, -7.2196e-01, -2.1118e+00,
           1.3255e-02, -2.2428e+00, -1.3125e-01, -2.2871e-01,  2.9346e-01,
          -1.2672e+00, -1.6063e+00,  4.9327e+00],
         [-3.8637e-01, -2.6075e+00, -1.7384e+00, -1.4457e+00, -1.1260e+00,
          -2.0737e+00, -2.8399e+00, -1.8835e+00, -1.5363e+00, -2.6820e+00,
          -2.0250e+00, -1.8139e+00, -1.4232e+00, -2.2700e+00, -2.0360e+00,
          -2.3014e+00, -3.2664e+00,  1.3467e+01],
         [-6.0329e-01, -2.0453e+00, -2.0059e+00, -1.5335e+00, -1.0740e+00,
          -1.9713e+00, -2.5437e+00, -2.2416e+00, -

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_ID = "Roblox/roblox-pii-classifier"
LOCAL_MODEL_DIR = "models_store/roblox"

if os.path.isdir(LOCAL_MODEL_DIR) and os.listdir(LOCAL_MODEL_DIR):
    print(f"Loading model from local directory: {LOCAL_MODEL_DIR}")
    tokenizer = AutoTokenizer.from_pretrained(LOCAL_MODEL_DIR, fix_mistral_regex=True)
    model = AutoModelForSequenceClassification.from_pretrained(LOCAL_MODEL_DIR)

else:
    print(f"Downloading model from Hugging Face Hub: {MODEL_ID}")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, fix_mistral_regex=True)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID, token=HF_TOKEN)

    tokenizer.save_pretrained(LOCAL_MODEL_DIR)
    model.save_pretrained(LOCAL_MODEL_DIR)

inputs = tokenizer(
    "can you give your email?", return_tensors="pt", truncation=True, is_split_into_words=False
)
print("Tokenized input IDs:", inputs["input_ids"])
print("Tokenized tokens:", tokenizer.convert_ids_to_tokens(inputs["input_ids"][0]))

with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
print("Model outputs:", outputs)
print("Logits:", logits)
print("Logits shape:", logits.shape)  

# Apply softmax to get probabilities
probs = torch.softmax(logits, dim=-1)[0]
print("Predicted probabilities:", probs)

# Map label IDs to actual label names
id2label = model.config.id2label
results = {id2label[i]: float(probs[i]) for i in range(len(probs))}
print("Label probabilities:", results)

# Cutoffs
combined_cutoff = 0.2691
asking_cutoff = 0.2
giving_cutoff = 0.3

# Combined cutoff check
max_prob = max(results.values())
is_pii_combined = max_prob >= combined_cutoff

# Individual cutoffs
is_pii_asking = results['p_privacy_asking_for_pii'] >= asking_cutoff
is_pii_giving = results['p_privacy_giving_pii'] >= giving_cutoff

# Final PII decision
is_pii_any = is_pii_asking or is_pii_giving

# Print results
print(f"\nText: {'what is your phone number?'}")
print(f"Max probability: {max_prob:.4f} → Combined cutoff ({combined_cutoff}) → PII detected? {is_pii_combined}")
print(f"Asking PII cutoff ({asking_cutoff}) → Detected? {is_pii_asking}")
print(f"Giving PII cutoff ({giving_cutoff}) → Detected? {is_pii_giving}")
print(f"Final decision (any threshold passed) → PII detected? {is_pii_any}")

Loading model from local directory: models_store/roblox
Tokenized input IDs: tensor([[   0,  831,  398, 8337,  935, 3340,  705,    2]])
Tokenized tokens: ['<s>', '▁can', '▁you', '▁give', '▁your', '▁email', '▁?', '</s>']
Model outputs: SequenceClassifierOutput(loss=None, logits=tensor([[ 2.7898, -3.7482]]), hidden_states=None, attentions=None)
Logits: tensor([[ 2.7898, -3.7482]])
Logits shape: torch.Size([1, 2])
Predicted probabilities: tensor([0.9986, 0.0014])
Label probabilities: {'p_privacy_asking_for_pii': 0.9985546469688416, 'p_privacy_giving_pii': 0.0014453405747190118}

Text: what is your phone number?
Max probability: 0.9986 → Combined cutoff (0.2691) → PII detected? True
Asking PII cutoff (0.2) → Detected? True
Giving PII cutoff (0.3) → Detected? False
Final decision (any threshold passed) → PII detected? True
